In [1]:
from scipy import stats as st
import numpy as np
import pandas as pd
import os
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
!pip install --upgrade xlrd
!pip install umap-learn
!pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 2.8 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.7 MB/s 
     |████████████████████████████████| 1.1 MB 33.1 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=fe49926a3294083f5cf259303ec8a9b0d5a634188761d75a7a0ffdfacb22d68c
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=c501f6b0a75ac864b7420b2cf8dbac3e0288c33c8f2f1bae982386fdb0277472
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dce

In [3]:
import zipfile
!rm Bin_data.zip
!wget https://github.com/vitvara/tobacco_project_review/raw/main/zip_file/Bin_data.zip
with zipfile.ZipFile("Bin_data.zip","r") as zip_ref:
    zip_ref.extractall()

rm: cannot remove 'Bin_data.zip': No such file or directory
--2022-06-17 03:43:41--  https://github.com/vitvara/tobacco_project_review/raw/main/zip_file/Bin_data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vitvara/tobacco_project_review/main/zip_file/Bin_data.zip [following]
--2022-06-17 03:43:41--  https://raw.githubusercontent.com/vitvara/tobacco_project_review/main/zip_file/Bin_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2118870 (2.0M) [application/zip]
Saving to: ‘Bin_data.zip’

Bin_data.zip        100%[===================>]   2.02M  --.-KB/s    in 0.06s   

2022-06-17 0

In [4]:
category_list = ["Burley", "Turkish", "Virginia"]
category_color = {"Burley":1, "Turkish":2, "Virginia":3}
all = []
c = []
samples = []
for category in category_list:
    for filename in os.listdir(os.path.join("Bin_data", category)):
        df = pd.read_excel(os.path.join("Bin_data", category, filename))
        samples.append(df)
        c.append(category_color[category])
        out = df.iloc[:, 1:].to_numpy().flatten("F").tolist()
        all.append(out)
        if len(out) != 11000:
            print(filename)

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

def test_raw_data(all, c):
    avgnural = 0
    avgrf = 0
    avglr = 0
    test_case = [0, 12, 34, 42, 75, 100, 150, 123, 343, 1034, 5643]
    highest = []
    lowest = [999999999]
    for i in test_case:
        print("random state:", i)
        X_train, X_test, y_train, y_test = train_test_split(all, c, test_size=0.33, random_state=i, stratify=c)

        reg = LogisticRegression(max_iter=100000000)
        rf = RandomForestClassifier(n_estimators=1000, random_state=0)
        clf = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(20,), random_state=42,max_iter=100000,
                    solver='lbfgs')
        clf.fit(X_train, y_train)
        a = clf.score(X_test, y_test)

        rf.fit(X_train, y_train)
        b = rf.score(X_test, y_test)

        reg.fit(X_train, y_train)
        c1 = reg.score(X_test,y_test)
        print("MLP RF LR scores:", a,b,c1)
        avgnural += a
        avgrf += b
        avglr += c1
        if sum(highest)/3 < sum([a,b,c1])/3:
            highest = [a,b,c1]
        if sum(lowest)/3 > sum([a,b,c1])/3:
            lowest = [a,b,c1]
    print(
        "avg MLP: ", avgnural/len(test_case)
    )
    print(
        "avg RF: ", avgrf/len(test_case)
    )
    print(
        "avg LR: ", avglr/len(test_case)
    )
    print("highest:", highest)
    print("lowest:", lowest)
    return (avgnural/len(test_case) + avgrf/len(test_case) + avglr/len(test_case))/3

In [63]:
def fea_select(data: list, n_feature: int, importance: list) -> list:
    f_i = list(zip(range(len(importance)),importance))
    f_i.sort(key = lambda x : x[1])
    out = []
    for row in data:
        out.append([row[i[0]] for i in f_i[-1:-n_feature:-1]])
    return out

def find_feature_importance(data: list, classes: list, importance: list) -> int:
    max = -9999999
    n_max_feature = None
    for i in [1000, 750, 500, 250, 100, 75, 50, 25, 10, 5]:
        reduced = fea_select(data, i, importance)
        np.array(reduced).shape
        result = test_raw_data(reduced, classes)
        if result > max:
            max = result
            n_max_feature = i
    print("N feature:", n_max_feature)
    target = n_max_feature
    max = -9999999
    n_max_feature = None
    for i in [target-10, target-5, target, target+5, target+10]:
        reduced = fea_select(data, i, importance)
        np.array(reduced).shape
        result = test_raw_data(reduced, classes)
        if result > max:
            max = result
            n_max_feature = i
    print(n_max_feature, "RESULT")
    n_max_feature
    max
    reduced = fea_select(data, n_max_feature, importance)
    np.array(reduced).shape
    result = test_raw_data(reduced, classes)
    return result

def get_feature(importance: list, n_features: int):
    f_i = list(zip(range(len(importance)),importance))
    f_i.sort(key = lambda x : x[1], reverse=True)
    return np.array(f_i)[:,0][:n_features]
    

In [7]:
feature_burley = pd.read_excel("FeatureSelectedBurley.xlsx")
feature_turkish = pd.read_excel("FeatureSelectedTurkish.xlsx")
feature_virginia = pd.read_excel("FeatureSelectedVirginia.xlsx")

In [78]:
feature_train = feature_virginia
selected = fea_select(all, 5000, feature_train.values.flatten())


In [79]:
x = 3
classes = []
for i in c:
    if i == x:
        classes.append(1)
    else:
        classes.append(0)

In [ ]:
test_raw_data(selected, classes)

In [90]:



rf = RandomForestClassifier(random_state=0)
rf.fit(selected,c)

RandomForestClassifier(random_state=0)

In [81]:
impor_selected = fea_select(all, , rf.feature_importances_)
test_raw_data(impor_selected, classes)

random state: 0
MLP RF LR scores: 0.5185185185185185 0.5555555555555556 0.5555555555555556
random state: 12


KeyboardInterrupt: ignored

In [70]:
all_feature = []

In [82]:
all_feature += get_feature(rf.feature_importances_, 5).tolist()

In [85]:
all_feature = list(set(all_feature))


In [88]:
selected = fea_select(all, 1000,all_feature)

In [89]:
test_raw_data(selected, c)

random state: 0
MLP RF LR scores: 0.3333333333333333 0.25925925925925924 0.4074074074074074
random state: 12
MLP RF LR scores: 0.4074074074074074 0.4074074074074074 0.37037037037037035
random state: 34
MLP RF LR scores: 0.2962962962962963 0.3333333333333333 0.4444444444444444
random state: 42
MLP RF LR scores: 0.5185185185185185 0.4074074074074074 0.2962962962962963
random state: 75
MLP RF LR scores: 0.4444444444444444 0.4074074074074074 0.4444444444444444
random state: 100
MLP RF LR scores: 0.3333333333333333 0.4444444444444444 0.48148148148148145
random state: 150
MLP RF LR scores: 0.25925925925925924 0.4074074074074074 0.4074074074074074
random state: 123
MLP RF LR scores: 0.4074074074074074 0.4074074074074074 0.4074074074074074
random state: 343
MLP RF LR scores: 0.37037037037037035 0.37037037037037035 0.4444444444444444
random state: 1034
MLP RF LR scores: 0.48148148148148145 0.4444444444444444 0.4444444444444444
random state: 5643
MLP RF LR scores: 0.37037037037037035 0.444444444

0.3973063973063973

In [91]:
find_feature_importance(selected, c, rf.feature_importances_)

random state: 0
MLP RF LR scores: 0.37037037037037035 0.25925925925925924 0.4074074074074074
random state: 12
MLP RF LR scores: 0.37037037037037035 0.4074074074074074 0.37037037037037035
random state: 34
MLP RF LR scores: 0.37037037037037035 0.3333333333333333 0.4444444444444444
random state: 42
MLP RF LR scores: 0.5185185185185185 0.4074074074074074 0.2962962962962963
random state: 75
MLP RF LR scores: 0.3333333333333333 0.4444444444444444 0.4444444444444444
random state: 100
MLP RF LR scores: 0.4074074074074074 0.48148148148148145 0.48148148148148145
random state: 150
MLP RF LR scores: 0.2962962962962963 0.4444444444444444 0.4074074074074074
random state: 123
MLP RF LR scores: 0.37037037037037035 0.2962962962962963 0.4074074074074074
random state: 343
MLP RF LR scores: 0.48148148148148145 0.37037037037037035 0.4444444444444444
random state: 1034
MLP RF LR scores: 0.4444444444444444 0.4444444444444444 0.4444444444444444
random state: 5643
MLP RF LR scores: 0.4074074074074074 0.4444444

0.40291806958473625

In [ ]:
25, 10, 5

In [ ]:
selected_feature

In [ ]:
from scipy.stats import ttest_ind

reduced = [[ row[i] for i,j in selected_feature] for row in data]
a = [sample for idx,sample in enumerate(reduced) if classes[idx] == 1 ]
b = [sample for idx,sample in enumerate(reduced) if classes[idx] == 2 ]
selected_idx = [idx for idx, pvalue in enumerate(ttest_ind(a=a, b=b).pvalue) if pvalue < 0.1]

In [ ]:
ttest_ind(a=a, b=b).pvalue

array([0.13585762, 0.10569111, 0.03861878, 0.27230923, 0.11122833,
       0.63485538, 0.06252678, 0.41235978, 0.12369468, 0.09344878,
       0.06092053, 0.30262569, 0.14984626, 0.06613013, 0.20056925,
       0.04809058, 0.03807582, 0.06664372, 0.18707519, 0.4571835 ,
       0.09193886, 0.47000161, 0.07041175, 0.1920363 , 0.65653366,
       0.63007217, 0.18768433, 0.10894077, 0.16506451, 0.8337846 ,
       0.10755884, 0.14733267, 0.2232018 , 0.82530129, 0.70342569,
       0.05562093, 0.61944177, 0.19745779, 0.11385358, 0.11234001,
       0.2599051 , 0.10187413, 0.29277806, 0.35602031, 0.09461389,
       0.17895267, 0.19417983, 0.76091038, 0.15596573, 0.0511738 ])

In [ ]:
len(selected_idx)

13

In [ ]:
reduced_ttest = [[ row[i] for i in selected_idx] for row in data]

In [ ]:
result = test_raw_data(reduced_ttest, classes)
print(result)

random state: 0
MLP RF LR scores: 0.3333333333333333 0.4 0.4666666666666667
random state: 12
MLP RF LR scores: 0.6 0.4666666666666667 0.4666666666666667
random state: 34
MLP RF LR scores: 0.4666666666666667 0.4666666666666667 0.4666666666666667
random state: 42
MLP RF LR scores: 0.4666666666666667 0.6 0.4666666666666667
random state: 75
MLP RF LR scores: 0.4 0.6 0.4666666666666667
random state: 100
MLP RF LR scores: 0.6666666666666666 0.6666666666666666 0.4666666666666667
random state: 150
MLP RF LR scores: 0.6 0.5333333333333333 0.4666666666666667
random state: 123
MLP RF LR scores: 0.5333333333333333 0.5333333333333333 0.4666666666666667
random state: 343
MLP RF LR scores: 0.6 0.5333333333333333 0.4666666666666667
random state: 1034
MLP RF LR scores: 0.5333333333333333 0.5333333333333333 0.4666666666666667
random state: 5643
MLP RF LR scores: 0.6666666666666666 0.5333333333333333 0.4666666666666667
avg MLP:  0.5333333333333333
avg RF:  0.5333333333333333
avg LR:  0.46666666666666673
